# Import

In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore', category=UserWarning, module='openpyxl')

ModuleNotFoundError: No module named 'plotly'

# 농업인 성별격차

Ref: [농림축산식품부 농업경영체 등록정보 현황서비스](https://edu.agrix.go.kr/uni_docs7/biOlap/dashBoard.do)

## 첫 번째 그래프

$\rightarrow$ 사용한 데이터

> 2015년부터 2023년까지의 농업경영체 현황(농업인)_지역별 농업인 현황 데이터

$\rightarrow$ 변수 설명

- 총 농업인 수
    - 전국의 농업인수를 합산하여 연도별 평균을 계산하였습니다.

- 성별 비율
    - 전국의 여성 및 남성 농업인수를 합산하여 성별로 총 농업인 수로 나누어 계산하였습니다.

$\rightarrow$ 데이터의 활용

총 농업인 수를 기준으로 농업인이 성별로 어떻게 다른 양상을 보이는지 확인하고자 하였습니다.

In [2]:
dfs = []

years = range(2015, 2024)

for year in years:
    file_path = f'./dataset/Farm_dataset/{year}_농업경영체 현황(농업인)_지역별 농업인 현황.csv'
    df = pd.read_csv(file_path, header=1).iloc[:-1, :4]
    df = df.rename(columns={'Unnamed: 0': '시도명', 'Unnamed: 1': '농업인수'})
    df = df.assign(남성비율 = round((df['남'] / df['농업인수'])*100, 2))
    df = df.assign(여성비율 = round((df['여'] / df['농업인수'])*100, 2))
    df = df.assign(연도 = year)
    dfs.append(df)
df_all = pd.concat(dfs, ignore_index=True)

In [3]:
df_avg = df_all.groupby('연도').agg({
    '남성비율': 'mean',
    '여성비율': 'mean'
}).reset_index()
fig = px.line(df_avg, x='연도', y=['남성비율', '여성비율'],
              title='농업인의 연도별 평균 남성비율과 여성비율',
              labels={'연도': '연도', 'value': '비율 (%)'},
              markers=True,
             width=800,height=400)
fig.update_layout(showlegend=True,legend_title_text="" ,
                  yaxis=dict(
                    range=[40, 60]  
    ),template='ggplot2')
fig

NameError: name 'px' is not defined

In [4]:
df_avg

,연도,남성비율,여성비율
0,2015,53.683529,46.275882
1,2016,54.349412,45.650588
2,2017,55.047647,44.952353
3,2018,55.345294,44.654706
4,2019,55.507647,44.492353
5,2020,55.585294,44.414706
6,2021,55.561176,44.438824
7,2022,55.555294,44.444706
8,2023,55.520588,44.479412


## 두 번째 그래프

$\rightarrow$ 사용한 데이터

> 2015년부터 2023년까지의 농업경영체 현황(농업인)_지역별 농업인 현황 데이터

$\rightarrow$ 변수 설명

- 총 농업인 수
    - 전국의 농업인수를 합산하여 연도별 평균을 계산하였습니다.

- 연령대 비율
    - 원본 데이터에는 **25세 미만, 25\~29세, 30\~34세, 35\~39세, 40\~44세, 45\~49세, 50\~54세, 55\~59세, 60\~64세, 65\~69세, 70\~74세, 75\~79세, 80세 이상**의 연령대가 있었습니다.
    - PPT내 연령대 표현을 통일화하기 위해서 **40대 이하, 50대, 60대, 70대 이상**의 4개의 항목으로 나타내었고, 아래와 같이 항목을 구분하였습니다.
        - 40대 이하 = 25세 미만 + 25\~29세 + 30\~34세 + 35\~39세 + 40\~44세 + 45\~49세
        - 50대 = 50\~54세 + 55\~59세
        - 60대 = 60\~64세 + 65\~69세
        - 70대 이상 = 70\~74세 + 75\~79세 + 80세 이상

$\rightarrow$ 데이터의 활용

총 농업인 수를 기준으로 농업인이 연령대별로 어떻게 다른 양상을 보이는지 확인하고자 하였습니다.

In [5]:
dfs = []

years = range(2015, 2024)
ages = ['40대 이하','50대','60대','70대 이상']

for year in years:
    file_path = f'./dataset/Farm_dataset/{year}_농업경영체 현황(농업인)_지역별 농업인 현황.csv'
    df = pd.DataFrame(pd.read_csv(file_path,header=1).iloc[-1,[1,2,3] + list(range(4, 17))]).T.rename(columns={'Unnamed: 1': '농업인수'}).apply(pd.to_numeric, errors='coerce')
    df = df.assign(남성비율 = round((df['남'] / df['농업인수'])*100, 2)).assign(여성비율 = round((df['여'] / df['농업인수'])*100, 2)).assign(연도 = year)
    df['40대 이하'] = df['25세 미만']+df['25~29세']+df['30~34세']+df['35~39세']+df['40~44세']+df['45~49세']
    df['50대'] = df['50~54세']+df['55~59세']
    df['60대'] = df['60~64세']+df['65~69세']
    df['70대 이상'] = df['70~74세']+df['75~79세']+df['80세 이상']
    df = df.apply(pd.to_numeric, errors='coerce')
    for age in ages:
        df[age + ' '] = round((df[age] / df['농업인수']) * 100, 2)
    dfs.append(df)
df_all = pd.concat(dfs, ignore_index=True).iloc[:,[16,17,18]+list(range(23,27))]

In [6]:
fig = px.line(df_all, x='연도', y=df_all.columns[2:], title='연도별 성별 및 연령대 비율',
              labels={'value': '비율', 'variable': '항목'},
              color_discrete_sequence=px.colors.qualitative.Set3,
             width = 800,height = 400)
fig.update_layout(template='ggplot2',legend_title_text="" ,
    legend=dict(
        orientation="h",
        yanchor="top",
        y=-0.2, 
        xanchor="center",
        x=0.5
    ))
fig.show()

NameError: name 'px' is not defined

In [7]:
df_all

,남성비율,여성비율,연도,40대 이하,50대,60대,70대 이상
0,52.56,47.40,2015,14.72,26.51,28.30,30.43
1,53.04,46.96,2016,13.53,25.75,29.33,31.39
2,53.76,46.24,2017,11.95,24.30,30.17,33.58
3,54.04,45.96,2018,11.10,23.41,30.75,34.74
4,54.24,45.76,2019,10.41,22.30,31.65,35.64
5,54.30,45.70,2020,9.99,20.93,32.76,36.33
6,54.21,45.79,2021,9.64,19.76,34.07,36.53
7,54.16,45.84,2022,9.20,18.74,34.43,37.63
8,54.12,45.88,2023,8.69,18.09,34.77,38.45


# 여성농업인 소득

Ref: [국가농식물통계서비스](https://kass.mafra.go.kr/newkass/kas/sti/sti/themeConfmStats.do?trg=001)

## 세 번째 그래프

$\rightarrow$ 사용한 데이터

> 2023년 여성농업인실태조사의 여성농업인의 농사일 담당 비중 데이터

$\rightarrow$ 변수 설명

- 평균 비율 
    - 원본 데이터에 존재하는 평균 데이터를 사용하였습니다.

$\rightarrow$ 데이터의 활용

연령대별 여성 농업인의 농사일 담당 비중을 제시하고자 하였습니다. 

비율 항목이 존재했지만, 여기서는 평균적인 여성 농업인의 농사일 담당 비중이 평균적으로 절반에 이르기 때문에 농업에서 여성의 역할을 명확히 이해할 수 있습니다.

In [8]:
df = pd.read_csv('dataset/Farm_dataset/2023_여성농업인의_농사일_담당_비중.csv',header=1).iloc[7:11,1:] 
df

,구분(2),사례수 (명),25%미만 (%),25%~50%미만 (%),50%~75%미만 (%),75% 이상 (%),평균 (%)
7,40대 이하,125,13.4,37.7,34.3,14.5,46.2
8,50대,296,4.7,31.6,44.1,19.6,55.4
9,60대,821,3.8,41.7,45.9,8.6,48.3
10,70대 이상,761,7.4,41.9,32.4,18.4,50.8


- 평균 비율의 평균

In [9]:
df.iloc[:,-1].mean()

50.175

In [10]:
age_groups = df['구분(2)']
fig = go.Figure()

fig.add_trace(go.Bar(
    x=age_groups,
    y=df['평균 (%)'],
    name=column
))
    
fig.update_layout(
    title='여성농업인의 농사일 담당 비중',
    xaxis_title='연령대',
    yaxis_title='평균 비율 (%)',
    barmode='stack',width=800,height=500,template='ggplot2',
    legend=dict(
        orientation="h",
        yanchor="top",
        y=-0.2, 
        xanchor="center",
        x=0.5
    ))
fig

NameError: name 'go' is not defined

## 네 번째 그래프

$\rightarrow$ 사용한 데이터

> 2023년 여성농업인실태조사의 농외 소득활동 희망 이유 데이터

$\rightarrow$ 변수 설명

- **농사만으로 생활비 부족 (%)** 항목의 데이터
    - 원본 데이터에 있는 항목의 데이터 그대로를 사용하였습니다.

$\rightarrow$ 데이터의 활용

세 번째 그래프에서 여성농업인의 농사일 담당 비중은 평균적으로 절반에 육박하지만, 농사만으로 생활비가 부족하여 농외 소득활동을 희망하는 여성의 비중이 작지 않다는 점을 강조하고자 하였습니다.

In [11]:
df = pd.read_csv('dataset/Farm_dataset/2023_농외_소득활동_희망_이유.csv',header=1).iloc[6:10,1:]
df

,구분(2),사례수 (명),농사만으로 생활비 부족 (%),일할 수 있을 때 추가소득 마련 (%),유휴시설을 활용하려고 (%),재능·장점을 활용하려고 (%),사회생활을 하려고 (%),타인 권유·정책적 지원 (%)
6,40대 이하,45,51.1,20.2,0.0,10.4,16.8,1.5
7,50대,75,45.0,40.1,0.0,5.8,9.2,0.0
8,60대,107,51.5,40.3,1.7,4.9,1.4,0.3
9,70대 이상,39,87.6,8.8,0.0,0.0,3.6,0.0


In [12]:
age_groups = df['구분(2)']
fig = go.Figure()

fig.add_trace(go.Bar(
    x=age_groups,
    y=df['농사만으로 생활비 부족 (%)']
))
    
fig.update_layout(
    title='농외 소득활동 희망 이유',
    xaxis_title='연령대',
    yaxis_title='비율 (%)',
    barmode='stack',width=800,height=500,template='ggplot2',
    legend=dict(
        orientation="h",
        yanchor="top",
        y=-0.2, 
        xanchor="center",
        x=0.5
    ))
fig

NameError: name 'go' is not defined

# 여성농업인 지위

Ref: [국가농식물통계서비스](https://kass.mafra.go.kr/newkass/kas/sti/sti/themeConfmStats.do?trg=001)

## 다섯 번째 그래프

$\rightarrow$ 사용한 데이터

> 2023년 여성농업인실태조사의 거주 지역사회에서 여성의 지위 인식 데이터

$\rightarrow$ 변수 설명

- 낮다, 같다, 높아졌다의 세 항목
    - 원본 데이터에는 **여전히 남성보다 낮다 (%), 예전보다 높아졌지만 아직 남성보다 낮다 (%), 남성과 같다 (%), 남성보다 높아졌다 (%), 잘 모르겠다 (%)**의 항목이 있었고,
    - 이를 아래와 같이 세 항목으로 나눠 표현하였습니다.
        - 낮다 = 여전히 남성보다 낮다 (%) + 예전보다 높아졌지만 아직 남성보다 낮다 (%)
        - 같다 = 남성과 같다 (%)
        - 높아졌다 = 남성보다 높아졌다 (%)

$\rightarrow$ 데이터의 활용

항목을 나눈 이유는 아래와 같습니다.

1. 분석을 단순화하여 핵심 경향을 명확히 파악할 수 있도록 하고
2.  남성과의 상대적 위치 변화를 직관적으로 비교할 수 있게 하며
3.  응답자가 확실히 알지 못하는 경우를 구분하여 전체 결과의 신뢰성을 높이기 위해서입니다.
    
여성 농업인의 상대적 지위 변화를 명확히 파악하고, 다양한 연령대에 따른 인식 차이를 확인하기 위해서 해당 데이터를 제시하였습니다.

In [13]:
df

,구분(2),사례수 (명),농사만으로 생활비 부족 (%),일할 수 있을 때 추가소득 마련 (%),유휴시설을 활용하려고 (%),재능·장점을 활용하려고 (%),사회생활을 하려고 (%),타인 권유·정책적 지원 (%)
6,40대 이하,45,51.1,20.2,0.0,10.4,16.8,1.5
7,50대,75,45.0,40.1,0.0,5.8,9.2,0.0
8,60대,107,51.5,40.3,1.7,4.9,1.4,0.3
9,70대 이상,39,87.6,8.8,0.0,0.0,3.6,0.0


- 연령별

In [14]:
df = pd.read_csv('dataset/Farm_dataset/2023_거주_지역사회에서_여성의_지위_인식.csv',header=1).iloc[7:11,1:] 

In [15]:
age_groups = df['구분(2)']
values = df.set_index('구분(2)').drop(columns='사례수 (명)')

values = values.assign(낮다 = values['여전히 남성보다 낮다 (%)']+ values['예전보다 높아졌지만 아직 남성보다 낮다 (%)'])
values = values.assign(같다 = values['남성과 같다 (%)'])
values = values.assign(높아졌다 = values['남성보다 높아졌다 (%)'])

values = values.iloc[:,5:]

In [16]:
fig = go.Figure()

for column in values.columns:
    fig.add_trace(go.Bar(
        x=age_groups,
        y=values[column],
        name=column
    ))
    
fig.update_layout(
    title='여성의 지위 인식 연령대별 비율',
    xaxis_title='연령대',
    yaxis_title='비율 (%)',
    barmode='stack',width=800,height=500,template='ggplot2',
    legend=dict(
        orientation="h",
        yanchor="top",
        y=-0.2, 
        xanchor="center",
        x=0.5
    ))
fig

NameError: name 'go' is not defined

# 여성농업인 역량

Ref: [국가농식물통계서비스](https://kass.mafra.go.kr/newkass/kas/sti/sti/themeConfmStats.do?trg=001)

## 여섯 번째 그래프

$\rightarrow$ 사용한 데이터

> 2023년 여성농업인실태조사의 디지털 기술 활용 역량 데이터

$\rightarrow$ 변수 설명

- 구분은 아래와 같으며, 원본 데이터를 사용하였습니다.
    - 스마트폰/PC 정보검색
    - 이메일/메신저 사용
    - 소셜미디어 서비스
    - 온라인 예약/쇼핑/결제
    - 온라인 공공행정 서비스
    - 온라인 금융거래
    - 키오스크 주문
    - 셀프주유소
- 항목은 아래와 같으며, 이 중 **혼자 능숙하게 사용함 (%)** 데이터를 사용하였습니다.
    - 해당 데이터만 시각화한 이유는 기술의 실제 활용 능력을 직접적으로 측정하기 위해서입니다. 이 데이터는 기술 사용의 자립성과 숙련도를 평가하는 데 중점을 두어, 실질적인 활용 수준을 파악할 수 있습니다.
    - 원본 데이터의 항목은 아래와 같습니다.
        - 전혀 사용할 줄 모름 (%)
        - 알지만 사용할 줄 모름 (%)
        - 사용할 수 있으나 도움 필요 (%)
        - 혼자 능숙하게 사용함 (%)
        - 비해당 (%)

$\rightarrow$ 데이터의 활용

연령대별로 '혼자 능숙하게 사용함' 응답 비율을 시각화하여 디지털 활용 능력의 차이를 보여주고자 하였습니다. 

이를 통해 각 연령대가 기술을 얼마나 자립적으로 활용할 수 있는지에 대한 명확한 비교가 가능합니다. 

이 정보는 연령대별 디지털 격차를 이해하고, 필요한 지원을 제공하는 데 유용합니다.

In [17]:
df = pd.read_csv('./dataset/Farm_dataset/2023_디지털_기술_활용_역량.csv',header=0).iloc[[0,1,9,10,11,12],2:].T.rename(columns={0:'구분',1:'항목',9:'40대 이하',10:'50대',11:'60대',12:'70대 이상'}).query('항목=="혼자 능숙하게 사용함 (%)"')
df['40대 이하'] = pd.to_numeric(df['40대 이하'], errors='coerce')
df['50대'] = pd.to_numeric(df['50대'], errors='coerce')
df['60대'] = pd.to_numeric(df['60대'], errors='coerce')
df['70대 이상'] = pd.to_numeric(df['70대 이상'], errors='coerce')

df = pd.melt(df, id_vars=['구분', '항목'], var_name='연령대', value_name='능숙도 (%)')

fig = px.line(df, x='구분', y='능숙도 (%)', color='연령대', markers=True,
              title='연령대별 혼자 능숙하게 사용함의 응답 비율',
             width=1000, height=600,template='ggplot2')

fig.show()

NameError: name 'px' is not defined

## 일곱 번째 그래프

$\rightarrow$ 사용한 데이터

> 2023년 여성농업인실태조사의 여성농업인의 자격증 보유_현황

$\rightarrow$ 변수 설명

- 평균 비율 
    - 원본 데이터에 존재하는 평균 (개) 데이터를 사용하였습니다.

$\rightarrow$ 데이터의 활용

연령대별 여성 농업인의 자격증 보유 현황을 제시하고자 하였습니다. 

연령대별로 명확한 차이를 보이는 자격증 보유 현황을 확인함으로써, 각 연령대의 전문성 및 교육 수준을 비교하고, 자격증 취득에 대한 지원 필요성을 파악할 수 있습니다.

In [18]:
df = pd.read_csv('./dataset/Farm_dataset/2023_여성농업인의_자격증_보유_현황.csv').iloc[7:11,[1,-1]].rename(columns={'구분(2)':'연령','2023.4':'자격증 평균 보유 수'})
df['자격증 평균 보유 수'] = pd.to_numeric(df['자격증 평균 보유 수'], errors='coerce')
fig = px.bar(df, x='연령', y='자격증 평균 보유 수',
             width=1000, height=600,template='ggplot2')
fig.show()

NameError: name 'px' is not defined